In [1]:
from pathlib import Path
from typing import List, Dict

from Models.LocalizedString import LocalizedString, ParamTypes, LocalizedStringParam
import Services.GoogleSheetsService as sheetService
from Services.LocalFilesService import LocalFilesService, isWindows

## Constants and configurations

In [2]:
filesBaseDir = Path("./CachedFiles")
filePathLocalizedStringRaw = filesBaseDir / "LocalizedStringRaw.json"
filePathLocalizedStringGroups = filesBaseDir / "LocalizedStringGroups.json"

fileService = LocalFilesService(baseFolder=filesBaseDir)

In [3]:
defaultLanguage = 'RU'
codeFileTab = '    '

filePathUniqueMessage = Path('../src/entities/bot-content/nested/unique-message.entity.ts')

In [4]:
cachedTrueValue = 'TRUE'

languagesStartSheetLetter = 'H'
languagesStartSheetLetterIndex = ord(languagesStartSheetLetter)

firstLetter = 'A'
firstLetterIndex = ord(firstLetter)

lastLetter = 'Z'
lastLetterIndex = ord(lastLetter)

# Starts from 1
configurationRowIndex = 1
startSheetLocalizationRowsIndex = 3
endSheetLocalizationRowsIndex = 1000

## Cache languages configuration

In [5]:
languages = sheetService.getContent(
    page=sheetService.pages.uniqueMessage,
    range=f'{languagesStartSheetLetter}{configurationRowIndex}:{lastLetter}{configurationRowIndex}'
)[0]

languagesCount = len(languages)
rowEstimatedLength = languagesStartSheetLetterIndex - firstLetterIndex + languagesCount
print(languages)
print(f'Languages count: {languagesCount}')
print(f'Estimated row length: {rowEstimatedLength}')

['RU', 'EN', 'AR', 'BE', 'ID']
Languages count: 5
Estimated row length: 12


In [6]:
def makeSpreadSheetPlaceholderWithIndex(index: int) -> str:
    return f'<#{index}>'

def makeIosPlaceholderWithIndex(index: int) -> str:
    return f'%{index}$@'

## Cache spreadsheet localization content

In [7]:
sheetStringsContent = sheetService.getContent(
    page=sheetService.pages.uniqueMessage, 
    range=f'{firstLetter}{startSheetLocalizationRowsIndex}:{chr(languagesStartSheetLetterIndex + languagesCount)}{endSheetLocalizationRowsIndex}'
)
sheetStringsContent = [row for row in sheetStringsContent if len(row) >= rowEstimatedLength]
len(sheetStringsContent)

136

In [8]:
def parseParams(rawParamsString: str) -> List[LocalizedStringParam]:
    result: List[LocalizedStringParam] = []
    for row in rawParamsString.split('\n'):
        rowComponents = [ component.strip() for component in row.split('/') ]
        if len(rowComponents) == 3 and rowComponents[2] in ParamTypes._member_names_:
            result.append(LocalizedStringParam(
                paramKey=rowComponents[0],
                name=rowComponents[1],
                paramType=ParamTypes[rowComponents[2]]
            ))
    return result

In [9]:
localizedStrings: List[LocalizedString] = []
for row in sheetStringsContent:
    localizedValues = row[rowEstimatedLength - languagesCount:]
    localizedValuesDict: Dict[str, str] = {}
    for index, localizedValue in enumerate(localizedValues):
        localizedValuesDict[languages[index]] = localizedValue

    localizedStrings.append(
        LocalizedString(
            group=row[0],
            key=row[1],
            comment=row[2],
            parameters=parseParams(row[3]),
            isUniqueMessage=row[4] == cachedTrueValue,
            localizedValues=localizedValuesDict
        )
    )
print(localizedStrings)


[LocalizedString(group='Sample', key='sample', comment='Тестовый текст', parameters=[], isUniqueMessage=False, localizedValues={'RU': 'Пример', 'EN': 'Example', 'AR': 'مثال', 'BE': 'Прыклад', 'ID': 'Contoh'}), LocalizedString(group='common', key='unknownState', comment='', parameters=[], isUniqueMessage=True, localizedValues={'RU': 'Я тебя не понимаю 😔\n⬇️ Нажми на кнопку снизу ⬇️', 'EN': "I don't understand you 😔\n⬇️ Click on the button below ⬇️", 'AR': 'لا أفهمك\n⬇️ اضغط على الزر أدناه ⬇️', 'BE': 'я цябе не разумею 😔\n⬇️ Націсні на кнопку знізу ⬇️', 'ID': 'Saya tidak mengerti kamu 😔\n⬇️ Klik tombol di bawah ⬇️'}), LocalizedString(group='common', key='permissionDenied', comment='', parameters=[], isUniqueMessage=True, localizedValues={'RU': 'У вас нет доступа к этому разделу', 'EN': 'You do not have access to this section', 'AR': 'ليس لديك حق الوصول إلى هذا القسم', 'BE': 'У вас няма доступу да гэтай часткі', 'ID': 'Anda tidak memiliki akses ke bagian ini'}), LocalizedString(group='com

## Unique messages generation

In [10]:
def classNameFrom(text: str) -> str:
    return text[0].upper() + text[1:]

def propertyNameFrom(text: str) -> str:
    return text[0].lower() + text[1:]

In [11]:
localizedGroups: Dict[str, List[LocalizedString]] = {}
for localizedString in localizedStrings:

    if localizedString.isUniqueMessage is False: continue
    
    if localizedString.group not in localizedGroups:
        localizedGroups[localizedString.group] = []
    localizedGroups[localizedString.group].append(localizedString)

In [12]:
def generateClassCodeForGroup(groupName: str, localizedValues: List[LocalizedString]) -> str:
    
    result = f'export class {classNameFrom(groupName)} '
    result += '{'

    for localizedString in localizedValues:
        
        comment = codeFileTab + '/**'
        comment += f'\n{codeFileTab} * {localizedString.localizedValues[defaultLanguage]}'
        comment += f'\n{codeFileTab} */'
        # result += f'\n{comment}'

        comm = f'\n    /** {localizedString.comment} */' if localizedString.comment else ''
        
        localizedValueSample = localizedString.localizedValues[defaultLanguage].replace('\n', '\\n')
        result += f'{comm}\n{codeFileTab}readonly {propertyNameFrom(localizedString.key)} = \'{localizedValueSample}\''
    
    result += '\n}'
    return result

In [13]:
uniqueMessagesFileContent = '''// ==================
// * Generated file *
// ==================

export class UniqueMessage {'''

for group in localizedGroups:
    uniqueMessagesFileContent += f'\n{codeFileTab}readonly {propertyNameFrom(group)} = new {classNameFrom(group)}()'

uniqueMessagesFileContent += '\n}\n'

for group, localizedString in localizedGroups.items():
    uniqueMessagesFileContent += f'\n{generateClassCodeForGroup(group, localizedString)}\n'

# print(uniqueMessagesFileContent)

In [14]:
import codecs
if isWindows:
    with codecs.open(filePathUniqueMessage, 'w', encoding='utf-8') as file:
        file.write(uniqueMessagesFileContent)
else:
    with filePathUniqueMessage.open('w') as file:
        file.write(uniqueMessagesFileContent)